In [1]:
import os

In [2]:
os.chdir('../')

In [3]:
%pwd

'c:\\Users\\anuru\\Desktop\\MyPro\\Chicken-Disease-Classification'

In [ ]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir:Path
    trained_model_path:Path
    updatd_base_model_path:Path
    training_data:Path
    params_epochs:int
    params_batch_size:int
    params_is_augmentation:bool
    params_image_size:list
    params_learning_rate: float

@dataclass(frozen=True)
class PrepareCallBackConfig:
    root_dir:Path
    tensorboard_root_log_dir:Path
    checkpoint_model_filepath:Path

In [5]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml,create_directories

In [ ]:
class ConfigurationManger:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_prepare_callback_config(self)->PrepareCallBackConfig:
        config=self.config.prepare_callbacks
        model_ckpt_dir=os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config=PrepareCallBackConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )
        return prepare_callback_config
    
    def get_training_config(self)->TrainingConfig:
        config=self.config.training
        training_data=os.path.join(
            self.config.data_ingestion.unzip_dir, "Chicken-fecal-images"
        )
        create_directories([
            Path(config.root_dir)
        ])

        training_config=TrainingConfig(
            root_dir=Path(config.root_dir),
            trained_model_path=Path(config.trained_model_path),
            updatd_base_model_path=Path(self.config.prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=self.params.EPOCHES,
            params_batch_size=self.params.BATCH_SIZE,
            params_is_augmentation=self.params.AGUMENATTION,
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE

        )

        return training_config

In [7]:
import time
import os 
import tensorflow as tf
tf.config.run_functions_eagerly(True)


In [8]:
class PrepareCallback:
    def __init__(self,config:PrepareCallBackConfig):
        self.config=config

    @property
    def _create_tb_callbacks(self):
        timestamp=time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir=os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}"
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )
    
    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [ ]:
class Training:
    def __init__(self, config:TrainingConfig):
        self.config=config

    def get_base_model(self):
        self.model=tf.keras.models.load_model(
            self.config.updatd_base_model_path,
            compile=False
        )

        self.model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=self.config.params_learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=['accuracy']

        )

    def train_valid_generator(self):
        datagenerator_kwargs=dict(
            rescale=1./255,
            validation_split=0.20
        )

        dataflow_kwargs=dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator= tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator=valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator=tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator=valid_datagenerator

        self.train_generator=train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path:Path, model:tf.keras.Model):
        model.save(path)

    def train(self,callback_list:list):
        self.steps_per_epoch=self.train_generator.samples//self.train_generator.batch_size
        self.validation_steps=self.valid_generator.samples//self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callback_list
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )




In [14]:
try:
    config=ConfigurationManger()
    prepare_callback_config=config.get_prepare_callback_config()
    prepare_callbacks=PrepareCallback(prepare_callback_config)
    callback_list=prepare_callbacks.get_tb_ckpt_callbacks()

    training_config=config.get_training_config()
    training=Training(training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callback_list=callback_list
    )
except Exception as e:
    raise e

[2025-12-18 12:47:52,122: INFO: common: Created dirctory at: artifacts]
[2025-12-18 12:47:52,125: INFO: common: Created dirctory at: artifacts\prepare_callbacks\checkpoint_dir]
[2025-12-18 12:47:52,127: INFO: common: Created dirctory at: artifacts\prepare_callbacks\tensorboard_root_log_dir]
[2025-12-18 12:47:52,129: INFO: common: Created dirctory at: artifacts\training]
Found 78 images belonging to 2 classes.
Found 312 images belonging to 2 classes.


c:\Users\anuru\Desktop\MyPro\Chicken-Disease-Classification\venv\lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.5598 - loss: 11.1244[2025-12-18 12:50:17,353: WARNING: saving_api: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]
19/19 ━━━━━━━━━━━━━━━━━━━━ 145s 8s/step - accuracy: 0.5709 - loss: 11.7983 - val_accuracy: 0.4688 - val_loss: 3.2777
[2025-12-18 12:50:17,462: WARNING: saving_api: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]
